# Equity Premium Prediction Analysis

This notebook follows the (Rapach 2010) [^1] to implement the prediction performance analysis.

[^1]: Rapach, D. E., Strauss, J. K., & Zhou, G. (2010). Out-of-Sample Equity Premium Prediction: Combination Forecasts and Links to the Real Economy. The Review of Financial Studies, 23(2), 821–862. https://doi.org/10.1093/rfs/hhp063


In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import sys
sys.path.append('../module')

from analysis import get_period_return
from analysis import get_oos_r_square,\
                     get_p_value_of_MSPE_adjusted_test,\
                     get_significance_of_MSPE_adjusted_test,\
                     get_utility_gain_from_prediction,\
                     get_significance_of_p_value
from data_handler import get_monthly_date_format
from data_handler import get_quarterly_date_format
from data_handler import get_econ_predictors
from IO_handler import post_dataframe_to_latex_table

%matplotlib inline

In (Rapach 2010), they adopt three measurements to evaluate the performance of the prediction of equity premium.

1. [$R^2_{OS}$: out of sample $R^2$](##out-of-sample-$r^2$)
2. [MSPE - adjusted statistic: The significance of the $R^2_{OS}$](##mspe-adjusted-test-(mean-squared-prediction-error))
3. [$\Delta$: The utility gain](##the-utility-gain)

load prediction results from linear regression (combination)

In [2]:
prediction_df = pd.read_csv('../../data/linear_prediction_quarterly.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

## out of sample $R^2$

$$
\begin{equation}
R_{O S}^2=1-\frac{\sum_{k=q_0+1}^q\left(r_{m+k}-\hat{r}_{m+k}\right)^2}{\sum_{k=q_0+1}^q\left(r_{m+k}-\bar{r}_{m+k}\right)^2}
\end{equation}
$$

Where $m$ is the size of in sample data and $q_0$ is the origin of the hold out period.

In [3]:
def get_oos_r_square(y_hat: np.ndarray, y: np.ndarray, y_bar: np.ndarray) -> float:
    """
    This function calculates the out-of-sample R square for a prediction.

    Parameters
    ----------
    y_hat : np.ndarray
        Prediction values.
    y : np.ndarray
        True values.
    y_bar : np.ndarray
        Historical average.

    Returns
    -------
    float
        Out-of-sample R square.
    """
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y_bar) ** 2)
    R_2 = 1 - ss_res / ss_tot
    R_2_percentage = R_2 * 100

    return R_2_percentage

In [5]:
R_2_OOS = prediction_ivv.apply(lambda x: get_oos_r_square(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
R_2_OOS

ivv_elastic    -7.786188
ivv_rf        -22.787563
dtype: float64

## MSPE adjusted test (Mean Squared Prediction Error)

We test whether a prediction method is significantly different from the historical average. We follow the statistical test by Clark and West (2007). Rapach (2010) also uses this statistical test method.

The null hypothesis is H0: $R_{OS}^2 < 0$

The test statistics is:
\begin{equation}
f_{t+1}=\left(r_{t+1}-\bar{r}_{t+1}\right)^2-\left[\left(r_{t+1}-\hat{r}_{t+1}\right)^2-\left(\bar{r}_{t+1}-\hat{r}_{t+1}\right)^2\right]
\end{equation}

We regress this statistics against a constant and get the one-side p-value.

In [6]:
def get_p_value_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> float:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    p_value_of_MSPE_adjusted : float
    """
    F = (y - y_bar) ** 2 - ((y - y_hat) ** 2 - (y_bar - y_hat) ** 2)
    dummy = np.ones_like(F)
    lm_result = sm.OLS(F, dummy).fit()
    p_value = lm_result.pvalues.values[0]

    return p_value


In [8]:
p_value_ivv = prediction_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
p_value_ivv

ivv_elastic    0.870105
ivv_rf         0.607642
dtype: float64

In [9]:
p_value_ief = prediction_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ief, y_hat=x, y_bar=historical_average_ief))
p_value_ief

ivv_elastic    0.231509
ivv_rf         0.938068
dtype: float64

In [10]:
def get_significance_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> str:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    significance_of_MSPE_adjusted : str
    """

    p_value = get_p_value_of_MSPE_adjusted_test(y=y, y_hat=y_hat, y_bar=y_bar)
    p_value = round(p_value, ndigits=3)
    if p_value >= 0.1:
        significance = str(p_value) + ' '
    elif p_value > 0.05:
        significance = str(p_value) +' *'
    elif p_value > 0.01:
        significance = str(p_value) +' **'
    elif p_value <= 0.01:
        significance = str(p_value) +' ***'

    return significance

In [9]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Dividend Price Ratio     0.756 
Dividend Yield           0.701 
Earnings Price Ratio     0.845 
Earnings Payout Ratio    0.189 
Stock Variance            0.35 
Book To Market           0.238 
Net Equity Expansion     0.417 
Treasury Bill            0.555 
Long Term Yield          0.365 
Long Term Return         0.128 
Term Spread              0.834 
Default Yield Spread     0.856 
Default Return Spread    0.229 
Inflation                0.383 
Mean                     0.958 
Median                   0.968 
Trimmed mean             0.969 
DMSPE theta 1            0.978 
DMSPE theta 0.9          0.394 
dtype: object

## The Utility Gain

We assume a mean-variance investor. He balance his portfolio between stock and risk-free bill monthly. The portfolio weights are decided by the prediction of the equity premium. The portfolio weights are given as

\begin{equation}
w_{j, t}=\left(\frac{1}{\gamma}\right)\left(\frac{\hat{r}_{t+1}}{\hat{\sigma}_{t+1}^2}\right)
\end{equation}

This weight is based on the prediction of the stock return and the variance of stock. We use ten years of rolling window for the estimation. This is in line with (Rapach 2010) and Campbell and Thompson (2008). The investor will gain an average utility over out-of-sample period as

\begin{equation}
\hat{v}_0=\hat{\mu}_0-\left(\frac{1}{2}\right) \gamma \hat{\sigma}_0^2
\end{equation}

To get the monthly portfolio weight we set $\gamma = 3$

In [109]:
def get_utility_gain_from_prediction(START_DATE: str,
                                     END_DATE: str,
                                     prediction: pd.DataFrame,
                                     historical_average: pd.DataFrame,
                                     equity_return: pd.DataFrame = None,
                                     rolling_window_size: int = 5, # number in year
                                     data_frequency: int = 12, # number of observations per year
                                     gamma: int = 3) -> float:
    """
    Get utility gain from prediction.
    TODO:
    -----
    1. the calculation of utility gain requires the true return of interested.
    2. we need to replace spy equity premium with the true equity return of interested.

    Parameters
    ----------
    START_DATE : str
        Start date of the utility gain curve.
        Format: YYYY-MM
    END_DATE : str
        End date of the utility gain curve.
        Format: YYYY-MM
    rolling_window_size : int, optional
        Rolling window size.
        Default: 10
    data_frequency : int, optional
        Data frequency.
        Default: 12
    gamma : int, optional
        Gamma.
        Default: 3

    Returns
    -------
    utility_gain : float
        Utility gain.
    """

    START_DATE = datetime.strptime(START_DATE, '%Y-%m')
    START_DATE = str(START_DATE.year - rolling_window_size) + '-' + str(START_DATE.month)
    econ_predictors = get_econ_predictors(START_DATE=START_DATE, END_DATE=END_DATE)
    risk_free_bond = econ_predictors['Treasury Bill'] / 100
    if equity_return is None:
        stock_return = econ_predictors['Equity Premium']
    else:
        stock_return = equity_return[START_DATE:END_DATE]
    portfolio_df = pd.concat([stock_return, risk_free_bond], axis=1).dropna()

    sample_varince = portfolio_df.iloc[:, 0].rolling(rolling_window_size * data_frequency - 1).var().dropna()
    varince_estimation = sample_varince.shift(1).dropna()

    stock_weight_0 = (1 / gamma) * (historical_average / varince_estimation)
    stock_weight_0 = stock_weight_0.clip(0, 1.5)
    portfolio_weight_0 = pd.concat([stock_weight_0, 1 - stock_weight_0], axis = 1)
    w_0 = portfolio_weight_0.values.reshape(-1, 1, 2)

    stock_weight_1 = (1 / gamma) * (prediction / varince_estimation)
    stock_weight_1 = stock_weight_1.clip(0, 1.5)
    portfolio_weight_1 = pd.concat([stock_weight_1, 1 - stock_weight_1], axis = 1)
    w_1 = portfolio_weight_1.values.reshape(-1, 1, 2)

    return_df = portfolio_df.loc[portfolio_weight_0.index] # need to change
    returns = return_df.values.reshape(-1, 2, 1)

    portfolio_return_0 = (w_0 @ returns).flatten()
    portfolio_return_1 = (w_1 @ returns).flatten()

    mu_0 = np.mean(portfolio_return_0)
    sigma_0 = np.var(portfolio_return_0)
    uitility_0 = mu_0 - 0.5 * gamma * sigma_0

    mu_1 = np.mean(portfolio_return_1)
    sigma_1 = np.var(portfolio_return_1)
    uitility_1 = mu_1 - 0.5 * gamma * sigma_1

    utility_gain = uitility_1 - uitility_0
    utility_gain_percentage = utility_gain * 100

    return utility_gain_percentage

In [99]:
utility_gain_ief = prediction_ief.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_ief.index[0].strftime('%Y-%m'), 
                                                                                    END_DATE=prediction_ief.index[-1].strftime('%Y-%m'),
                                                                                    historical_average=historical_average_ief, 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_ief))
utility_gain_ief

ief_elastic   -0.103964
ief_rf        -0.200901
dtype: float64

In [12]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Dividend Price Ratio,-5.320067,0.756,-0.210819
Dividend Yield,-7.202495,0.701,-0.181242
Earnings Price Ratio,-0.521225,0.845,-0.040984
Earnings Payout Ratio,-5.026098,0.189,-0.485624
Stock Variance,-27.288147,0.35,-0.274800
Book To Market,-2.305251,0.238,-0.369258
Net Equity Expansion,-0.309303,0.417,-0.116939
Treasury Bill,-2.549524,0.555,-0.205161
Long Term Yield,-2.001667,0.365,-0.168449
Long Term Return,2.858106,0.128,0.454339


In [176]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='equity premium out-of-sample forecasting')

Save table to:../../table/


# Analysis of linear prediction

load prediction results from linear regression (combination)

In [2]:
prediction_df_ivv = pd.read_csv('../../data/linear_prediction_ivv_2012_2019.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
prediction_df_ief = pd.read_csv('../../data/linear_prediction_ief_2012_2019.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
equity_return_ivv = prediction_df_ivv.pop('Equity Premium')
equity_return_ief = prediction_df_ief.pop('Equity Premium')
historical_average_ivv = prediction_df_ivv.pop('Historical Average')
historical_average_ief = prediction_df_ief.pop('Historical Average')

1. R2

In [3]:
R_2_OOS_ivv = prediction_df_ivv.apply(lambda x: get_oos_r_square(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
R_2_OOS_ief = prediction_df_ief.apply(lambda x: get_oos_r_square(y=equity_return_ief, y_hat=x, y_bar=historical_average_ief))
R_2_OOS = pd.concat([R_2_OOS_ivv, R_2_OOS_ief], axis=1)
R_2_OOS = R_2_OOS.mean(axis=1)
R_2_OOS.name = 'Linear R2'
R_2_OOS

Dividend Price Ratio     0.321484
Dividend Yield           1.019365
Earnings Price Ratio    -0.725371
Earnings Payout Ratio   -0.260041
Stock Variance           1.911039
Book To Market          -0.877066
Net Equity Expansion    -1.739989
Treasury Bill            0.328626
Long Term Yield          2.761571
Long Term Return         0.937281
Term Spread             -0.028410
Default Yield Spread     0.446151
Default Return Spread   -0.172786
Inflation               -2.820003
Mean                     0.855329
Median                   0.322169
Trimmed mean             0.655975
DMSPE theta 1            0.834393
DMSPE theta 0.9          1.072366
Name: Linear R2, dtype: float64

2. significance of R2

In [10]:
p_value_ivv = prediction_df_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
p_value_ief = prediction_df_ief.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ief, y_hat=x, y_bar=historical_average_ief))
p_value = pd.concat([p_value_ivv, p_value_ief], axis=1)
p_value = p_value.mean(axis=1)
significance = {key:get_significance_of_p_value(value) for key, value in p_value.to_dict().items()}
significance = pd.DataFrame(significance, index=['Linear Significance']).T
significance

,Linear Significance
Dividend Price Ratio,0.021 **
Dividend Yield,0.097 *
Earnings Price Ratio,0.143
Earnings Payout Ratio,0.346
Stock Variance,0.313
Book To Market,0.586
Net Equity Expansion,0.654
Treasury Bill,0.524
Long Term Yield,0.091 *
Long Term Return,0.515


The utility gain

In [25]:
IVV_IEF_daily_ret = pd.read_csv('../../data/IVV_IEF_daily_ret.csv', index_col=0, parse_dates=True)
IVV_IEF_monthly_ret = IVV_IEF_daily_ret.groupby(pd.Grouper(freq='M')).apply(get_period_return)
IVV_IEF_monthly_ret.index = [get_monthly_date_format(x) for x in IVV_IEF_monthly_ret.index]
equity_return_ivv = IVV_IEF_monthly_ret['IVV']
equity_return_ief = IVV_IEF_monthly_ret['IEF']

In [31]:
utility_gain_ivv = prediction_df_ivv.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df_ivv.index[0].strftime('%Y-%m'), 
                                                                                      END_DATE=prediction_df_ivv.index[-1].strftime('%Y-%m'),
                                                                                      historical_average=historical_average_ivv, 
                                                                                      prediction=x,
                                                                                      equity_return=equity_return_ivv))
utility_gain_ief = prediction_df_ief.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df_ief.index[0].strftime('%Y-%m'), 
                                                                                      END_DATE=prediction_df_ief.index[-1].strftime('%Y-%m'),
                                                                                      historical_average=historical_average_ief, 
                                                                                      prediction=x,
                                                                                      equity_return=equity_return_ief))
utility_gain = pd.concat([utility_gain_ivv, utility_gain_ief], axis=1)
utility_gain = utility_gain.mean(axis=1)
utility_gain.name = 'Linear Utility Gain'
utility_gain

Dividend Price Ratio     0.006817
Dividend Yield           0.025233
Earnings Price Ratio    -0.023961
Earnings Payout Ratio   -0.012437
Stock Variance           0.037661
Book To Market           0.020104
Net Equity Expansion    -0.144362
Treasury Bill            0.031324
Long Term Yield          0.088271
Long Term Return        -0.010193
Term Spread             -0.010767
Default Yield Spread     0.022526
Default Return Spread    0.009855
Inflation               -0.033999
Mean                     0.017366
Median                   0.013264
Trimmed mean             0.013425
DMSPE theta 1            0.017247
DMSPE theta 0.9          0.018717
Name: Linear Utility Gain, dtype: float64

In [32]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
# evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,Linear R2,Linear Significance,Linear Utility Gain
Dividend Price Ratio,0.321484,0.021 **,0.006817
Dividend Yield,1.019365,0.097 *,0.025233
Earnings Price Ratio,-0.725371,0.143,-0.023961
Earnings Payout Ratio,-0.260041,0.346,-0.012437
Stock Variance,1.911039,0.313,0.037661
Book To Market,-0.877066,0.586,0.020104
Net Equity Expansion,-1.739989,0.654,-0.144362
Treasury Bill,0.328626,0.524,0.031324
Long Term Yield,2.761571,0.091 *,0.088271
Long Term Return,0.937281,0.515,-0.010193


In [33]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='prediction performance of linearly predicting ivv and ief return')

Save table to:../../table/


# Analysis of ML prediction

load ml prediction results

In [63]:
prediction_df = pd.read_csv('../../data/prediction_of_equity_return_ivv_ief_ml.csv', 
                            index_col=0,
                            parse_dates=True,
                            date_parser=get_monthly_date_format)
IVV_IEF_daily_ret = pd.read_csv('../../data/IVV_IEF_daily_ret.csv', index_col=0, parse_dates=True)
IVV_IEF_monthly_ret = IVV_IEF_daily_ret.groupby(pd.Grouper(freq='M')).apply(get_period_return)
IVV_IEF_monthly_ret.index = [get_monthly_date_format(x) for x in IVV_IEF_monthly_ret.index]
equity_return_total_ivv = IVV_IEF_monthly_ret['IVV']
equity_return_total_ief = IVV_IEF_monthly_ret['IEF']
prediction_ivv = prediction_df[['ivv_elastic', 'ivv_rf']]
prediction_ief = prediction_df[['ief_elastic', 'ief_rf']]
historical_average_ivv = prediction_df['ivv_average']
historical_average_ief = prediction_df['ief_average']
equity_return_ivv = prediction_df['ivv_return']
equity_return_ief = prediction_df['ief_return']

R2

In [56]:
R_2_OOS_ief = prediction_ief.apply(lambda x: get_oos_r_square(y=prediction_df['ief_return'], y_hat=x, y_bar=historical_average_ief))
R_2_OOS_ivv = prediction_ivv.apply(lambda x: get_oos_r_square(y=prediction_df['ivv_return'], y_hat=x, y_bar=historical_average_ivv))
R_2_OOS_ivv.index = ['Elastic Net', 'Random Forest']
R_2_OOS_ief.index = ['Elastic Net', 'Random Forest']
R_2_OOS = pd.concat([R_2_OOS_ivv, R_2_OOS_ief], axis=1)
R_2_OOS = R_2_OOS.mean(axis=1)
R_2_OOS.name = 'ML R2'
R_2_OOS

Elastic Net      -5.538171
Random Forest   -19.026376
Name: ML R2, dtype: float64

significance of R2

In [69]:
p_value_ivv = prediction_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
p_value_ief = prediction_ief.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ief, y_hat=x, y_bar=historical_average_ief))
p_value_ivv.index = ['Elastic Net', 'Random Forest']
p_value_ief.index = ['Elastic Net', 'Random Forest']
p_value = pd.concat([p_value_ivv, p_value_ief], axis=1)
p_value = p_value.mean(axis=1)
significance = {key:get_significance_of_p_value(value) for key, value in p_value.to_dict().items()}
significance = pd.DataFrame(significance, index=['ML Significance']).T
significance

,ML Significance
Elastic Net,0.847
Random Forest,0.622


The utility gain

In [72]:
utility_gain_ief = prediction_ief.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_ief.index[0].strftime('%Y-%m'), 
                                                                                    END_DATE=prediction_ief.index[-1].strftime('%Y-%m'),
                                                                                    historical_average=historical_average_ief, 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ief))
utility_gain_ivv = prediction_ivv.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_ivv.index[0].strftime('%Y-%m'), 
                                                                                    END_DATE=prediction_ivv.index[-1].strftime('%Y-%m'),
                                                                                    historical_average=historical_average_ivv, 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ivv))
utility_gain_ivv.index = ['Elastic Net', 'Random Forest']
utility_gain_ief.index = ['Elastic Net', 'Random Forest']
utility_gain = pd.concat([utility_gain_ivv, utility_gain_ief], axis=1)
utility_gain = utility_gain.mean(axis=1)
utility_gain.name = 'ML Utility Gain'
utility_gain

Elastic Net      0.013332
Random Forest   -0.102459
Name: ML Utility Gain, dtype: float64

In [73]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Elastic Net,-5.538171,0.847,0.013332
Random Forest,-19.026376,0.622,-0.102459


In [74]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='prediction performance of ML predicting ivv and ief return')

Save table to:../../table/


# Analysis of implied expected return

load implied expected return

In [99]:
implied_return_mktcap = pd.read_csv('../../data/implied_expected_return_MKTCAP.csv', index_col=0, parse_dates=True)
implied_return_equal = pd.read_csv('../../data/implied_expected_return_equal.csv', index_col=0, parse_dates=True)

implied_return_equal.index = [get_monthly_date_format(x) for x in implied_return_equal.index]
implied_return_mktcap.index = [get_monthly_date_format(x) for x in implied_return_mktcap.index]

implied_return_total_ivv = pd.concat([implied_return_mktcap['IVV'], implied_return_equal['IVV']], axis=1)
implied_return_total_ivv.columns = ['MKTCAP', 'Equal Weight']
implied_return_total_ief = pd.concat([implied_return_mktcap['IEF'], implied_return_equal['IEF']], axis=1)
implied_return_total_ief.columns = ['MKTCAP', 'Equal Weight']

START_DATE = '2007-09'
END_DATE = '2019-11'
implied_return_ief = implied_return_total_ief[START_DATE:END_DATE]
implied_return_ivv = implied_return_total_ivv[START_DATE:END_DATE]

R2

In [114]:
R_2_OOS_ief = implied_return_ief.apply(lambda x: get_oos_r_square(y=equity_return_total_ief[START_DATE:END_DATE], y_hat=x, y_bar=historical_average_ief[START_DATE:END_DATE]))
R_2_OOS_ivv = implied_return_ivv.apply(lambda x: get_oos_r_square(y=equity_return_total_ivv[START_DATE:END_DATE], y_hat=x, y_bar=historical_average_ivv[START_DATE:END_DATE]))
R_2_OOS_ivv.index = ['MKTCAP', 'Equal Weight']
R_2_OOS_ief.index = ['MKTCAP', 'Equal Weight']
R_2_OOS = pd.concat([R_2_OOS_ivv, R_2_OOS_ief], axis=1)
R_2_OOS = R_2_OOS.mean(axis=1)
R_2_OOS.name = 'Implied R2'
R_2_OOS

MKTCAP         -2.173875
Equal Weight   -2.195446
Name: Implied R2, dtype: float64

Significance of R2

In [119]:
p_value_ivv = implied_return_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_total_ivv[START_DATE:END_DATE], y_hat=x, y_bar=historical_average_ivv[START_DATE:END_DATE]))
p_value_ief = implied_return_ief.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_total_ief[START_DATE:END_DATE], y_hat=x, y_bar=historical_average_ief[START_DATE:END_DATE]))
p_value_ivv.index = ['MKTCAP', 'Equal Weight']
p_value_ief.index = ['MKTCAP', 'Equal Weight']
p_value = pd.concat([p_value_ivv, p_value_ief], axis=1)
p_value = p_value.mean(axis=1)
significance = {key:get_significance_of_p_value(value) for key, value in p_value.to_dict().items()}
significance = pd.DataFrame(significance, index=['Implied Significance']).T
significance

,Implied Significance
MKTCAP,0.701
Equal Weight,0.693


The utility gain

In [120]:
utility_gain_ief = implied_return_ief.apply(lambda x: get_utility_gain_from_prediction(START_DATE=START_DATE, 
                                                                                    END_DATE=END_DATE,
                                                                                    historical_average=historical_average_ief[START_DATE:END_DATE], 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ief))
utility_gain_ivv = implied_return_ivv.apply(lambda x: get_utility_gain_from_prediction(START_DATE=START_DATE, 
                                                                                    END_DATE=END_DATE,
                                                                                    historical_average=historical_average_ivv[START_DATE:END_DATE], 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ivv[START_DATE:END_DATE]))
utility_gain_ivv.index = ['MKTCAP', 'Equal Weight']
utility_gain_ief.index = ['MKTCAP', 'Equal Weight']
utility_gain = pd.concat([utility_gain_ivv, utility_gain_ief], axis=1)
utility_gain = utility_gain.mean(axis=1)
utility_gain.name = 'Implied Utility Gain'
utility_gain

MKTCAP         -0.278634
Equal Weight   -0.271653
Name: Implied Utility Gain, dtype: float64

Output results

In [121]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
MKTCAP,-2.173875,0.701,-0.278634
Equal Weight,-2.195446,0.693,-0.271653


In [122]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='prediction performance of implied return of ivv and ief')

Save table to:../../table/
